# подключим библиотеки

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from rep.utils import train_test_split_group
from sklearn.cross_validation import train_test_split
from pandas.tools.plotting import radviz,scatter_matrix,bootstrap_plot,parallel_coordinates
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib
#‘nbagg’, ‘notebook’, inline,‘osx’, ‘qt’, ‘qt4’, ‘qt5’, ‘tk’, ‘wx’). If given, the corresponding matplotlib backend is used, otherwise it will be matplotlib’s default (which you can set in your matplotlib config file).

Using matplotlib backend: Qt4Agg


# пропишем функции

In [ ]:
def validation(X,y,size):
    l=round(size*len(X))
    Xtr=X[0:(l-1)]
    Xval=X[(l):len(X)]
    ytr=y[0:(l-1)]
    yval=y[(l):len(X)]
    
    return Xtr,ytr,Xval,yval

def create_cv(id_event, n_folds):
    id_unique = id_event.unique()
    size_fold = int(len(id_unique)/n_folds)
    idx_cv = []

    for i in range(n_folds):
        mask = id_event.isin(id_unique[i*size_fold : (i+1)*size_fold])
        idx_train = id_event.index[~mask].values
        idx_test = id_event.index[mask].values
        idx_cv.append([idx_train, idx_test])

    return idx_cv   

def compute_mean(event_ids, values):
    """ fore each event computes average of given values """
    number_of_sv_in_event = np.bincount(event_ids)
    return np.bincount(event_ids, weights=values) / number_of_sv_in_event

In [ ]:
train = pd.read_csv("data/training.csv")
test = pd.read_csv("data/test.csv")
subm = pd.read_csv("data/submission.csv")

In [ ]:
train['size'] = train.groupby(['EventID'])['Mass'].transform('count') 
test['size'] = test.groupby(['EventID'])['Mass'].transform('count') 


In [ ]:
lab1 = train[train['Label']==1]
lab0 = train[train["Label"]==0]

In [ ]:
len(lab0['size'].unique())

In [ ]:
ev=lab1.EventID
trains =lab1.groupby('EventID',as_index=False)

di=list(set('qwertyuiopasdfghjklzxcvb'))
trainn=trains.nth(0)
trainn['keys'] = np.arange(len(trainn))
trainn.head()


In [ ]:
traink=trains.nth(2)
traink.columns[1:16]+'i'

In [ ]:
traink.columns = 

In [ ]:
z=['EventID']
z.extend(traink.columns[1:17]+'i')
traink.columns = z

In [ ]:

traink.columns = z


In [ ]:
traink.columns

In [ ]:
len(lab1[lab1['Flight_distance']>1000])


In [ ]:
res=trainn.append(traink)
res.head()

In [ ]:
z = pd.merge(traink,trainn,on='EventID')
z.head()

In [ ]:
z.columns

In [ ]:
z = pd.concat([traink,trainn],axis=1,join_axes=traink['key'])
#z['EventID'] =ev
z.head()

In [ ]:
train.columns

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
train.drop(['EventID','Weight','size'],axis=1,inplace=True)

In [ ]:
y = train.Label
train.drop(['Label'],axis=1,inplace=True)

In [ ]:
Xtr, Xval, ytr, yval = train_test_split(train, y, test_size=0.20, random_state=42)
from sklearn.cluster import KMeans

In [ ]:

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


from sklearn.cluster import KMeans
from sklearn import datasets

np.random.seed(5)

centers = [[1, 1], [-1, -1], [1, -1]]

X = Xtr


estimators = {'k_means_iris_3': KMeans(n_clusters=3),
              'k_means_iris_8': KMeans(n_clusters=8),
              'k_means_iris_bad_init': KMeans(n_clusters=3, n_init=1,
                                              init='random')}


fignum = 1
for name, est in estimators.items():
    fig = plt.figure(fignum, figsize=(4, 3))
    plt.clf()
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

    plt.cla()
    est.fit(X)
    labels = est.labels_

    ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=labels.astype(np.float))

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel('Petal width')
    ax.set_ylabel('Sepal length')
    ax.set_zlabel('Petal length')
    fignum = fignum + 1

# Plot the ground truth
fig = plt.figure(fignum, figsize=(4, 3))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

plt.cla()

for name, label in [('Setosa', 0),
                    ('Versicolour', 1),
                    ('Virginica', 2)]:
    ax.text3D(X[y == label, 3].mean(),
              X[y == label, 0].mean() + 1.5,
              X[y == label, 2].mean(), name,
              horizontalalignment='center',
              bbox=dict(alpha=.5, edgecolor='w', facecolor='w'))
# Reorder the labels to have colors matching the cluster results
y = np.choose(y, [1, 2, 0]).astype(np.float)
ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=y)

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_xlabel('Petal width')
ax.set_ylabel('Sepal length')
ax.set_zlabel('Petal length')
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
y_pred = KMeans(n_clusters=3, random_state=7).fit_predict(Xtr)

plt.subplot(222)
plt.scatter(Xtr[:, 5], Xtr[:, 6], c=y_pred)
plt.show()

In [ ]:
Xtr.columns

In [ ]:
losses=[]
ks = np.arange(1,15,1)
for k in ks:
    knn = KNeighborsClassifier(weights='distance', n_neighbors=k)
    knn.fit(Xtr, ytr)
    losses.append(roc_auc_score(yval, knn.predict_proba(Xval)))
    print(losses)

In [ ]:
trainn.columns=[trainn.columns[] + di[k]]

In [ ]:
trainn.columns

In [ ]:
vl.columns[1:14]=[vl.columns[1:14]]

In [ ]:
train["size"].max()

In [ ]:
from sklearn.metrics import roc_auc_score

In [7]:
def baes(data,test,k,size_train,tree):
    di=list(set('qwertyuiopasdfghjklzxcvbnm1234567890'))
    prob = 0
    s = 0
    auc = 0
    prob_test = 0
    training_data, validation_dataa = train_test_split_group(data.EventID, data, random_state=7, train_size=size_train)
    lab1 = training_data[training_data['Label']==1]
    lab0 = training_data[training_data["Label"]==0]
    validation_dataa['tracks_dif']=validation_dataa.Tracks_number-validation_dataa.Tracks_number_passed
    #validation_data=validation_data.groupby('EventID', as_index=False).mean()
    daq=pd.DataFrame()
    for i in range(1,200):
        print("Model for",i)
        if(i<k):
            validation_dataq = validation_dataa[validation_dataa['size']==i]
            validation_dataq.drop(['size'],axis=1,inplace=True)
            data_1 = lab1[lab1["size"] ==i]
            trainnn=pd.concat([data_1,lab0])
            trainnn.drop(['size'],axis=1,inplace=True)
            trainnn['tracks_dif']=trainnn.Tracks_number-trainnn.Tracks_number_passed
            trainn=trainnn.groupby('EventID', as_index=False).mean()
            
            validation_data=validation_dataq.groupby('EventID', as_index=False).mean()
            print(trainn.shape,validation_data.shape)
            #test['tracks_dif']=test.Tracks_number-test.Tracks_number_passed
            trains =trainnn.groupby('EventID',as_index=False)
            #trains.drop(['Weight','Label'],axis=1,inplace=True)
            validation_datas = validation_dataq.groupby('EventID',as_index=False)
            #validation_datas.drop(['Weight','Label'],axis=1,inplace=True)
            #validation_data = validation_datas.nth(0)
            #trainn=trains.nth(0)
            if (i>1):
                for te in range(0,i):
                    z=['EventID']
                    vl = validation_datas.nth(te)
                    vl.drop(['Weight','Label'],axis=1,inplace=True)
                    z.extend(vl.columns[1:15]+di[te])
                    vl.columns=z
                    #validation_data = pd.concat([validation_data,vl],axis=1)
                    validation_data = pd.merge(validation_data,vl,on='EventID')
                    tr = trains.nth(te)
                    tr.drop(['Weight','Label'],axis=1,inplace=True)
                    tr.columns=z
                    trainn = pd.merge(trainn,tr,on='EventID')
                    
                    #trainn = pd.concat([trainn,tr],axis=1)
            #print(trainn.head())
            #trainn=trainn.groupby('EventID', as_index=False).mean()
            #trainn=trainn.groupby('EventID', as_index=False).mean()

            #test=test.groupby('EventID', as_index=False).mean()

            print(trainn.shape)
            
            features = list(set(trainn.columns) - {'EventID', 'Label', 'Weight','size'})
            
            y=trainn.Label
            print((1-y.mean())/y.mean(),y.mean(),sum(y))
            model = xgb.XGBClassifier(max_depth= 7,learning_rate= 0.015,nthread=4,subsample= 0.8,objective='binary:logistic',colsample_bytree= 0.9,min_child_weight=0,n_estimators=tree,seed=1337,scale_pos_weight=(1-y.mean())/y.mean())
            model.fit(trainn[features], trainn.Label)
            #kaggle_proba = model.predict_proba(test[features])
            proba = model.predict_proba(validation_data[features])

            events_ids = np.unique(validation_data.EventID)
            
            # compute number of SVs in each event
            number_of_sv_in_event = np.bincount(validation_data.EventID)

            # compute predictions for events (take the mean value of predictions for SVs forming an event)
            events_proba = compute_mean(validation_data.EventID, proba[:, 1])[events_ids]
            
           
            #prob = prob + (1/k)*events_proba
            # compute weights for events 
            events_weights = compute_mean(validation_data.EventID, validation_data.Weight)[events_ids]
            
            # compute labels for events 
            events_labels = compute_mean(validation_data.EventID, validation_data.Label)[events_ids]
            daqa = pd.DataFrame()
            daqa['EventID']=events_ids
            daqa['proba'] = events_proba
            daqa['Weight'] = events_weights
            daqa['Label'] = events_labels
            print(daqa.shape)
            a = roc_auc_score(events_labels, events_proba, sample_weight=events_weights)
            #print(roc_auc_score(events_labels, prob, sample_weight=events_weights))
            auc = auc +a
            s=s+i
            daq=pd.concat([daq,daqa])
            print(a)
            #prob_test = prob_test+(1/k)*kaggle_proba
            
        if(i==k):
            validation_data = validation_dataa[validation_dataa['size']>(k-1)]
            validation_data.drop(['size'],axis=1,inplace=True)
            validation_data=validation_data.groupby('EventID', as_index=False).mean()
            data_1 = lab1[lab1["size"] >(k-1)]
            trainn=pd.concat([data_1,lab0])
            trainn['tracks_dif']=trainn.Tracks_number-trainn.Tracks_number_passed

            test['tracks_dif']=test.Tracks_number-test.Tracks_number_passed
            trainn.drop(['size'],axis=1,inplace=True)
            trainn=trainn.groupby('EventID', as_index=False).mean()

            #test=test.groupby('EventID', as_index=False).mean()

            print(trainn.shape)
            features = list(set(trainn.columns) - {'EventID', 'Label', 'Weight','size'})
            y=trainn.Label
            print((1-y.mean())/y.mean(),y.mean(),sum(y))
            model = xgb.XGBClassifier(max_depth= 7,learning_rate= 0.015,nthread=4,subsample= 0.8,objective='binary:logistic',colsample_bytree= 0.9,min_child_weight=0,n_estimators=tree,seed=1337,scale_pos_weight=(1-y.mean())/y.mean())
            model.fit(trainn[features], trainn.Label)
            #kaggle_proba = model.predict_proba(test[features])
            proba = model.predict_proba(validation_data[features])
            events_ids = np.unique(validation_data.EventID)

            # compute number of SVs in each event
            number_of_sv_in_event = np.bincount(validation_data.EventID)

            # compute predictions for events (take the mean value of predictions for SVs forming an event)
            events_proba = compute_mean(validation_data.EventID, proba[:, 1])[events_ids]

            #prob = prob + (1/k)*events_proba
            # compute weights for events 
            events_weights = compute_mean(validation_data.EventID, validation_data.Weight)[events_ids]

            # compute labels for events 
            events_labels = compute_mean(validation_data.EventID, validation_data.Label)[events_ids]
            daqa = pd.DataFrame()
            daqa['EventID']=events_ids
            daqa['proba'] = events_proba
            daqa['Weight'] = events_weights
            daqa['Label'] = events_labels
            daq=pd.concat([daq,daqa])
            a = roc_auc_score(events_labels, events_proba, sample_weight=events_weights)
            #print(roc_auc_score(events_labels, prob, sample_weight=events_weights))
            auc = auc +a
            s=s+i
            print(a)
            #prob_test = prob_test+(1/k)*kaggle_proba
            break
    print( roc_auc_score(daq.Label, daq.proba, sample_weight=daq.Weight))
    #print(auc*(1/5))
    #print("prob",prob*(1/s))
    return 1#prob_test*(1)
    

In [8]:
pr = baes(train,test,25,0.8,1000)

/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Model for 1
(14376, 17) (3120, 17)
(14376, 17)
0.43846307784670796 0.6951864218141347 9994.0
(3120, 4)
0.954242981975
Model for 2
(10561, 17) (1746, 17)


/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(7890, 45)
0.2769056481631332 0.7831432192648923 6179.0
(1746, 4)
0.943189576762
Model for 3
(14206, 17) (2600, 17)


/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(445, 283)
0.0933660933660934 0.9146067415730337 407.0
(88, 4)
0.981678670758
Model for 20
(4740, 17) (93, 17)


/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dz/.local/lib/python3.5/site-packages/ipykernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(393, 297)
0.09776536312849157 0.910941475826972 358.0
(93, 4)


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
data_1 = lab1[lab1["size"] ==1]
data_1.shape 

In [ ]:
len(data_1[data_1.Label ==1])

In [ ]:
outcome = train.Weight

outcome.value_counts(normalize=True)
outcome.value_counts().plot(kind='pie', figsize=(5,5))

In [ ]:
len(train[train['Mass']<100])

In [ ]:
len(test)

In [ ]:
train.drop(['EventID','Track_number_PV','Tracks_number','Tracks_number_passed','Weight','Flight_distance'],axis=1,inplace=True)

In [ ]:
lab0 = train[train.Label == 0]
lab1 = train[train.Label == 1]

In [ ]:
lab0.describe()

In [ ]:
lab1.describe()

In [ ]:
lab1[lab1.Pseudorapidity < lab1.Pseudorapidity.min()+3*0.562888].count()

In [ ]:
#data.runtime[data.runtime==0] = np.nan
lab0[[ 'Mass', 'Corrected_mass', 'Pt', 'Pt_sum', 'Pt_min',
       'IP_chi2', 'IP_chi2_sum', 'Flight_distance', 'Pseudorapidity',
       'Track_number_PV', 'Tracks_number', 'Tracks_number_passed',
       'Vertex_chi2','size']].describe()

In [ ]:
lab1[['Mass', 'Corrected_mass', 'Pt', 'Pt_sum', 'Pt_min',
       'IP_chi2', 'IP_chi2_sum', 'Flight_distance', 'Pseudorapidity',
       'Track_number_PV', 'Tracks_number', 'Tracks_number_passed',
       'Vertex_chi2', 'size']].describe()

In [ ]:
train.Label.sum()

In [ ]:
train.columns

In [ ]:
train.drop(['EventID','Track_number_PV','Tracks_number','Tracks_number_passed','Weight','Flight_distance'],axis=1,inplace=True)

In [ ]:
lab0 = train[train.Label == 0]
lab1 = train[train.Label == 1]

In [ ]:
#lab0.drop(['Label'],axis=1,inplace=True)
#lab1.drop(['Label'],axis=1,inplace=True)

In [ ]:
from numpy import genfromtxt, mean, std
import scipy.stats as stats

In [ ]:
col = train.columns

for c in col:
    z_statistic, p_value = stats.wilcoxon(lab0[c] - lab1[c])
    print(c,lab0[c].mean(),lab0[c].std(),lab1[c].mean(),lab1[c].std(),p_value)
#scatter_matrix(train, alpha=0.2, figsize=(6, 6), diagonal='kde');

In [ ]:
col = train.columns

for c in col:
    print(c,lab0[c].min(),lab1[c].min())
print(col)

In [ ]:
len(lab0)

In [ ]:
#lab1[lab1['Corrected_mass']>25000].Corrected_mass
plt.figure();
#train['Mass'].diff().hist(color='k', alpha=0.5, bins=50,figsize=(200, 200));
lab0['Flight_distance'].hist(color='k', alpha=1, bins=100,figsize=(9, 9))
plt.show()

In [ ]:
plt.figure();
lab1.diff().hist(color='k', alpha=0.5, bins=100,figsize=(100, 100));
plt.show()

In [ ]:
lab0.columns

In [ ]:
#train = train.take(np.random.permutation(len(train)))

In [ ]:
train['Test']= False
test['Test']=True

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
data = pd.concat([train,test],ignore_index = True)


In [ ]:
data['z'] = data.Tracks_number_passed

In [ ]:
data = pd.get_dummies(data, columns = ['Tracks_number_passed'])#,'Tracks_number'])#,'Tracks_number_passed'])['Tracks_number']

In [ ]:
data.head()

In [ ]:
train.shape

In [ ]:
import random
random(100)

In [ ]:
train.Track_number_PV.unique(),train.Tracks_number.unique(),train.Tracks_number_passed.unique()

In [ ]:
test.Track_number_PV.unique(),test.Tracks_number.unique(),test.Tracks_number_passed.unique()

In [ ]:
x=[1,100,3,4,50,6,7,8,90,10]
stats.shapiro(x)

In [ ]:
import scipy.stats as stats
#scipy.stats.shapiro(x, a=None, reta=False)[source]
y=train.Label
for s in train.columns:
    z1=s
    print(z1,stats.anderson(train[z1]))

In [ ]:
train.columns

In [ ]:
train_new = pd.concat([])

In [ ]:
train = data[data.Test == 0].drop('Test', axis=1).copy()
test = data[data.Test == 1].drop('Test', axis=1).copy()

In [ ]:
trainn=pd.concat([data_1,lab0])
print(trainn.shape)
trainn.head()

In [ ]:
#Xtr,ytr,Xval,yval = validation(train,y,0.8)
#create_cv(id_event, n_folds)
features = list(set(trainn.columns) - {'EventID', 'Label', 'Weight','size'})
print(features)
training_data, validation_data = train_test_split_group(trainn.EventID, trainn, random_state=7, train_size=0.8)
training_data.shape,validation_data.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=300, n_jobs=4)


In [ ]:
model.fit(training_data[features], training_data.Label)

In [ ]:
y=trainn.Label
(1-y.mean())/y.mean(),y.mean(),sum(y)

In [ ]:
model = xgb.XGBClassifier(max_depth= 6,learning_rate= 0.025,nthread=4,subsample= 0.8,objective='binary:logistic',colsample_bytree= 0.9,min_child_weight=0,
                       n_estimators=1000,seed=1337,scale_pos_weight=(1-y.mean())/y.mean())

In [ ]:
model.fit(training_data[features], training_data.Label,early_stopping_rounds=100)

In [ ]:
params = {
    'max_depth': 7,
    'eta': 0.015,
    'nthread': 4,
    'subsample': 0.8,
    'eval_metric': ['auc'],
    'objective': 'binary:logistic',
    'colsample_bytree': 0.9,
    'min_child_weight': 0,
    'scale_pos_weight':(1-y.mean())/y.mean(),
    'seed':1337
}

In [ ]:
dtr = xgb.DMatrix(training_data[features], label=training_data.Label)
dval = xgb.DMatrix(validation_data[features], label=validation_data.Label)
watchlist = [(dtr, 'train'), (dval, 'eval')]
history = dict()
model = xgb.train(params, dtr, num_boost_round=2000, evals=watchlist,evals_result=history, verbose_eval=10)

In [ ]:
def compute_mean(event_ids, values):
    """ fore each event computes average of given values """
    number_of_sv_in_event = np.bincount(event_ids)
    return np.bincount(event_ids, weights=values) / number_of_sv_in_event

In [ ]:
validation_data[features].head()

In [ ]:
proba = model.predict(dval)
proba

In [ ]:
events_ids = np.unique(validation_data.EventID)

# compute number of SVs in each event
number_of_sv_in_event = np.bincount(validation_data.EventID)

# compute predictions for events (take the mean value of predictions for SVs forming an event)
events_proba = compute_mean(validation_data.EventID, proba[:])[events_ids]

# compute weights for events 
events_weights = compute_mean(validation_data.EventID, validation_data.Weight)[events_ids]

# compute labels for events 
events_labels = compute_mean(validation_data.EventID, validation_data.Label)[events_ids]

In [ ]:
def compute_mean(event_ids, values):
    """ fore each event computes average of given values """
    number_of_sv_in_event = np.bincount(event_ids)
    return np.bincount(event_ids, weights=values) / number_of_sv_in_event
dtest = xgb.DMatrix(test[features])

In [ ]:
proba = model.predict_proba(validation_data[features])
events_ids = np.unique(validation_data.EventID)

# compute number of SVs in each event
number_of_sv_in_event = np.bincount(validation_data.EventID)

# compute predictions for events (take the mean value of predictions for SVs forming an event)
events_proba = compute_mean(validation_data.EventID, proba[:, 1])[events_ids]

# compute weights for events 
events_weights = compute_mean(validation_data.EventID, validation_data.Weight)[events_ids]

# compute labels for events 
events_labels = compute_mean(validation_data.EventID, validation_data.Label)[events_ids]

In [ ]:
roc_auc_score(events_labels, events_proba, sample_weight=events_weights)

In [ ]:
y=trainn.Label
(1-y.mean())/y.mean(),y.mean(),sum(y)

In [ ]:
test.columns

In [ ]:
training_data[features], training_data.Label

In [ ]:
dtest = xgb.DMatrix(test[features])

In [ ]:
len(pr),len(subm)

In [ ]:
kaggle_proba = model.predict_proba(test[features])

In [ ]:
len(kaggle_proba)

In [ ]:
#kaggle_proba = model.predict_proba(test[features])
kaggle_proba = pr
kaggle_ids = np.unique(test.EventID)
# compute predictions for events (take the mean value of predictions for SVs forming an event)
kaggle_events_proba = compute_mean(test.EventID, kaggle_proba[:, 1])[kaggle_ids]

In [ ]:
len(kaggle_proba[kaggle_ids]),len(kaggle_events_proba)

# usually mean

In [3]:
x =pd.read_csv('xgbmean700full.csv')
y = pd.read_csv('baesSizeFeatch20minmaxformorekfulldata.csv')
#z = pd.read_csv('baesSizeFeatch20minmax.csv')
l = pd.read_csv('xgbmin-mean.csv')
#a = pd.read_csv('nwtmean.csv')
g = pd.read_csv('gbt.csv')
r =pd.read_csv('rf1000iso3.csv')

In [4]:
r.head()

,EventID,Label
0,0,0.080462
1,1,0.997553
2,2,0.969521
3,3,0.974991
4,4,0.105194


In [2]:
import pandas as pd

In [6]:
s = pd.merge(x,y,on='EventID')
s.head()

,EventID,Label_x,Label_y
0,0,0.005918,0.008036
1,1,0.995567,0.998945
2,2,0.837679,0.970941
3,3,0.884182,0.993898
4,4,0.009715,0.046013


In [7]:
import scipy.stats as stats
x=s.Label_x
y=s.Label_y
corr = {}l
corr['pearson'], _ = stats.pearsonr(x,y)
corr['spearman'], _ = stats.spearmanr(x,y)
corr['kendall'], _ = stats.kendalltau(x,y)
corr

SyntaxError: invalid syntax (<ipython-input-7-19da76df17bc>, line 4)

In [8]:
g['Label'] = s.Label_x+s.Label_y+l.Label+r.Label
g.to_csv('mean.csv', index=False)
g.head()

,EventID,Label
0,0,0.098019
1,1,3.987851
2,2,3.648814
3,3,3.757560
4,4,0.169390


In [42]:
x.head()

,EventID,Label
0,0,0.005918
1,1,0.995567
2,2,0.837679
3,3,0.884182
4,4,0.009715


# rank mean

In [ ]:
x =pd.read_csv('xgbmean700full.csv')
y = pd.read_csv('bayes.csv')
array = np.array(x.Label)
temp = array.argsort()
ranksx = np.arange(len(array))[temp.argsort()]
array = np.array(y.Label)
temp = array.argsort()
ranksb = np.arange(len(array))[temp.argsort()]
ranksb

In [ ]:
predsk = ranksb+ranksx
y['Label'] = predsk
y.to_csv('meanbayes+xgbmean(rank).csv', index=False)
y.head()

In [ ]:
z = pd.read_csv('meanbayes+xgbmean.csv')
z.head()

In [ ]:
prb=pr[:,1]

In [ ]:
probas = pd.DataFrame()
probas = x.Label+pr[:,1]
probas.head()

In [ ]:
from IPython.display import FileLink
def create_solution(ids, proba, filename='nwtmean.csv'):
    """saves predictions to file and provides a link for downloading """
    pd.DataFrame({'EventID': ids, 'Label': proba}).to_csv('{}'.format(filename), index=False)
    return FileLink('{}'.format(filename))
    
create_solution(kaggle_ids, kaggle_events_proba)

In [ ]:
array = np.array(prx)
temp = array.argsort()
ranksx = np.arange(len(array))[temp.argsort()]
array = np.array(prb)
temp = array.argsort()
ranksb = np.arange(len(array))[temp.argsort()]
ranksb

In [ ]:
prob_mean = p